As the name of dataset says ( "all-trumps-twitter-insults" ) this dataset just contain those tweets which Donal J. Trump wasn't so Polite :D .Let's see what was the problem and who made him angry :)

<br>

this dataset is so much similar to [this comp](https://www.kaggle.com/c/tweet-sentiment-extraction). we can assume insult part is same as selected text in the comp. 

if you were in that competition feel free to show your EDA's here too.


I inspired by many notebooks there such as :
* https://www.kaggle.com/tanulsingh077/twitter-sentiment-extaction-analysis-eda-and-model/
* https://www.kaggle.com/rahul253801/eda-of-trump-s-twitter-insults
* https://www.kaggle.com/shahules/complete-eda-baseline-model-0-708-lb

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


from collections import Counter

import spacy
from spacy.lang.en import English


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


from plotly.offline import download_plotlyjs, plot, init_notebook_mode, iplot
init_notebook_mode(connected=True)# initiate notebook for offline plot
import plotly.graph_objs as go
import os
import plotly.express as px


from sklearn.feature_extraction.text import CountVectorizer


from nltk.corpus import stopwords
import plotly.offline as py
import nltk
from nltk.corpus import stopwords
py.init_notebook_mode(connected=True)
nltk.download('stopwords')
stop=set(stopwords.words('english'))


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tweets_csv = pd.read_csv('/kaggle/input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv')
tweets_csv.drop(columns=['Unnamed: 0'], inplace=True)
tweets_csv['date']= pd.to_datetime(tweets_csv['date'])
tweets_csv.shape

In [ ]:
# create meta data
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


tweets_csv['jacard_score'] = tweets_csv.apply(lambda x: jaccard(x['tweet'], x['insult']), axis=1)

tweets_csv['tweet_num_word'] = tweets_csv['tweet'].apply(lambda x : len(str(x).split()))
tweets_csv['insult_num_word'] = tweets_csv['insult'].apply(lambda x : len(str(x).split()))
tweets_csv.head(5)

# Let's check jacard score and same rows first

In [ ]:
tweets_csv.head(5)

focuse on first and second rows. they are same tweets. but 2 parts were selected as insult. BUT second row has jacard_score 0.03 while first row has 0.0, WHY?

if you look closer, insult for first row was 'fool' but we have 'fool,' in text of the tweet. So this is the reason. maybe need a better function to calculte jacard. but for now forget about this problem.

Let's check how many of our rows are same Tweets with different insult part


** TIP ** : I know trump love CAPSLOCK and tweet many times in uppercase but I will lower case all the words.

In [ ]:
duplicate = tweets_csv[tweets_csv.duplicated(['tweet', 'date'])] 
duplicate['tweet'] = duplicate['tweet'].str.lower()
duplicate['insult'] = duplicate['insult'].str.lower()
duplicate.shape, tweets_csv.shape

Half of data are same in tweet and date ( means they are one tweet but different insults are there )it means we have just 4678 unique tweets.

# Insult length

Let's see how many word of a tweet were insults.

In [ ]:
plt.figure(figsize=(12, 6))
p1 = sns.kdeplot(tweets_csv['tweet_num_word'], shade = True, color='r').set_title('Kernel dirtribution of number of words')
p1 = sns.kdeplot(tweets_csv['insult_num_word'], shade=True, color='b')

most of the time insults are just smething around 8 words of tweet. So he is not so Rude :)

In [ ]:
plt.figure(figsize=(12, 6))
p1 = sns.kdeplot(tweets_csv['jacard_score'], shade = True, color='r').set_title('Kernel dirtribution of jacard score')

We expect to see jacard under 0.5 but we see a peak around 1. Weird? So let's see what are those?

In [ ]:
tweets_csv[tweets_csv['jacard_score'] > 0.8]

So As we can see these are the tweets which their insult part and tweet part are some how same.

Let's see these tweets target.

In [ ]:
target_full_insult = tweets_csv[tweets_csv['jacard_score'] > 0.8]
plt.figure(figsize=(12,6))
plt.xticks(rotation=30)
g = sns.countplot(x='target',data=target_full_insult, order=target_full_insult.target.value_counts().iloc[:9].index)


# Targets

Let's see who made him angry:D. first let's see how many target we have

In [ ]:
tweets_csv.target.nunique()

So there are 866 target. Let's see how many times trump tweeted against them.

In [ ]:
tweets_csv.target.value_counts()

In [ ]:
trace = go.Bar(x=tweets_csv.target.value_counts().index[:25], y=tweets_csv.target.value_counts(),
              marker=dict(
                  opacity=0.8,
                  color=np.arange(25)
              ))

fig = go.Figure(data=[trace])
fig.update_layout(title="Top 25 Targets")
fig.update_xaxes(title="Target")
fig.update_yaxes(title="Frequency")

iplot(fig)

So As we can remember he had a lot of conflicts with Media and democrate.

# Let's see Trump's vocabulary

## most common words in tweets

Let's show most common words, but first we need 2 steps : 

1. ignore duplicated rows ( we did it before )
2. remove stop words

In [ ]:
nlp = spacy.load("en")
# Let's add some stop word to list.
nlp.Defaults.stop_words |= {"The","&", "-", "A"}
def remove_stop(row):
    return [word for word  in row if word not in nlp.Defaults.stop_words]

duplicate['tweet_removed_stop_word'] = duplicate['tweet'].apply(lambda x:str(x).split()).apply(lambda x:remove_stop(x))
duplicate['insult_removed_stop_word'] = duplicate['insult'].apply(lambda x:str(x).split()).apply(lambda x:remove_stop(x))

In [ ]:
top_words = Counter([item for row in duplicate['tweet_removed_stop_word'] for item in row])
top_wrods_df = pd.DataFrame(top_words.most_common(20))
top_wrods_df.columns = ['word', 'count']
top_wrods_df.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(top_wrods_df, x="count", y="word", title='Most words in tweets', orientation='h', 
             width=700, height=700,color='word')
fig.show()

In [ ]:
fig = px.treemap(top_wrods_df, path=['word'], values='count',title='Tree Of Most Common words in tweets')
fig.show()

## most common words in insults

In [ ]:
top_words = Counter([item for row in duplicate['insult_removed_stop_word'] for item in row])
top_wrods_df = pd.DataFrame(top_words.most_common(20))
top_wrods_df.columns = ['word', 'count']
top_wrods_df.style.background_gradient(cmap='Reds')

In [ ]:
fig = px.bar(top_wrods_df, x="count", y="word", title='Most Commmon Words in insult', orientation='h', 
             width=700, height=700,color='word')
fig.show()

In [ ]:
fig = px.treemap(top_wrods_df, path=['word'], values='count',title='Tree Of Most Common Words in insults')
fig.show()

As we can see Fake, is so much repeated. As you can see he is not so rude :).

he had so much attention to election.

But my question is how insult part is selected ?

## Word cloud in insults and tweets

In [ ]:
pos_mask = np.array(Image.open('/kaggle/input/masksforwordclouds/twitter_mask4.jpg'))
stopwords = set(STOPWORDS)
more_stopwords = {'I', "i"}
stopwords = stopwords.union(more_stopwords)

wordcloud = WordCloud(background_color='white',
                    stopwords = stopwords,
                    max_words = 200,
                    max_font_size = 100, 
                    random_state = 42,
                    width=400, 
                    height=200,
                    mask = pos_mask)
wordcloud.generate(str(duplicate.tweet))
plt.figure(figsize=(16.0,9.0))
plt.imshow(wordcloud)
plt.title('word cloud for tweets', fontdict={'size': 16,  
                                  'verticalalignment': 'bottom'})

In [ ]:
pos_mask = np.array(Image.open('/kaggle/input/masksforwordclouds/twitter_mask4.jpg'))
stopwords = set(STOPWORDS)
more_stopwords = {'I', "i"}
stopwords = stopwords.union(more_stopwords)

wordcloud = WordCloud(background_color='white',
                    stopwords = stopwords,
                    max_words = 200,
                    max_font_size = 100, 
                    random_state = 42,
                    width=400, 
                    height=200,
                    mask = pos_mask)
wordcloud.generate(str(duplicate.insult))
plt.figure(figsize=(16.0,9.0))
plt.imshow(wordcloud)
plt.title('word cloud for insult  ', fontdict={'size': 16,  
                                  'verticalalignment': 'bottom'})

#  Check Rare words

I didn't focus on text_cleaning so much and you can see some punctutation and other things there. You can clean your data more and get better results here.

This will be useful for you to double check your text dataset by this approach

In [ ]:

def plot_n_first_rare_word(main_df, column, from_index, to_index):
    '''
    main_df : is dataframe you want to check
    column : is the column you want to check it's rare words
    from_index, to_index :  you will set a range to see
    for example if you set from_index : 200 and to index : 300  you will see range 200 to 300
    '''
    if to_index < from_index: assert print("from_index must be greater than from_index")
    rare_words = Counter([item for row in main_df[column] for item in row])
    rare_words_df = pd.DataFrame(rare_words.most_common()[-to_index:-from_index])
    rare_words_df.columns = ['word', 'count']
    fig = px.treemap(rare_words_df, path=['word'], values='count',title=f'plot for range of {from_index} to {to_index} rare words')
    fig.show()



In [ ]:
plot_n_first_rare_word(duplicate, "tweet_removed_stop_word",100, 200)

# Check ngrams

Checking ngrams are so useful. Always check them

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=set(stopwords.words('english'))
py.init_notebook_mode(connected=True)

def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n),stop_words=stop).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:20]


In [ ]:
new=tweets_csv['tweet']
top_n_bigrams=get_top_ngram(new,2)[:20]
pd.DataFrame(top_n_bigrams, columns=['n-gram','count']).set_index('n-gram').plot(kind='bar');

In [ ]:
new=tweets_csv['tweet']
top_n_bigrams=get_top_ngram(new,3)[:20]
pd.DataFrame(top_n_bigrams, columns=['n-gram','count']).set_index('n-gram').plot(kind='bar');

this was our plots for ngrams

# a little work with time

In [ ]:
datecount_data = tweets_csv.date.value_counts().resample("M").count()
trace1 = go.Scatter(x=datecount_data.index, y=datecount_data)
fig = go.Figure(data=[trace1])

fig.update_xaxes(title="Year")
fig.update_yaxes(title="Tweet Count")

iplot(fig)